In [ ]:
from sys import version, version_info
assert version_info >= (3, 7), "Need python >= 3.7, you have:\n{}".format(version_info)

# %load_ext blackcellmagic
# Usage: %%black

from dataclasses import dataclass, field
from typing import List
from queue import Queue, Empty
from random import shuffle, choice

import ipywidgets as widgets
from IPython.display import display, clear_output
import asyncio
from time import sleep

# Enable widgets:
# !jupyter nbextension install --py widgetsnbextension --user
# !jupyter nbextension enable widgetsnbextension --user --py

In [ ]:
@dataclass
class Pairs:
    japanese: str
    english: str
    

@dataclass
class Player:
    name: str
    vocab: List[Pairs] = field(repr=False)
#     known: List[Pairs] = field(default_factory=lambda :[], init=False)
        
    def __post_init__(self):
        shuffle(self.vocab)
        
        self.to_check = Queue()
        for i in self.vocab:
            self.to_check.put(i)
            
        self.score = {
            "known": 0,
            "failed": 0,
            "skipped": 0,
        }
        
    def update(self, category):
        self.score[category] += 1
       
    def pick_word(self):
        choice = self.to_check.get_nowait()
        return choice
    
    def show_score(self):
        for k in sorted(self.score.keys()):
            print("{:<10}: {:>3}/{:>3} [{:>4.1%}]".format(
                k, self.score[k], len(self.vocab), self.score[k] / len(self.vocab)))

# _raw = []
# with open("./previous_words.txt", "r") as f:
#     for i in f.readlines():
#         _raw.append(i[:-1])

# pairs_previous = [
#     Pairs(english=eng, japanese=jap)
#     for (eng, jap) in [x.split(" - \n", maxsplit=1) for x in _raw]
# ]

_raw = []
# with open("./new_words.txt", "r") as f:
idx = 0
with open("./sentences.txt", "r") as f:
    _buff = []
    for i in f.readlines():
        if len(i) < 4:
#             idx -= 1
            continue
        _buff.append(i[:-1].split(" -", maxsplit=1)[0])
#         print(idx % 3)
        
        if ((idx + 1) % 3) == 0 and idx != 0:
            _raw.append(" - ".join(_buff))
#             print(idx, _raw[-1])
            _buff = []
        idx += 1

pairs_new = [
    Pairs(english=eng, japanese=jap)
    for (eng, jap) in [x.split(" - ", maxsplit=1) for x in _raw]
]

In [ ]:
%gui asyncio

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # TODO: learn how to use futures proprely
        try:
            future.result()
        except asyncio.InvalidStateError:
            future.set_result(0)
    widget.on_click(getvalue)
    return future

In [ ]:
NEW = True
# NEW = False
_players = ["Dan", "Romeo", "Paul"]

if NEW:
    pairs = pairs_new
    vocab_size = len(pairs) * len(_players)
else:
    pairs = pairs_previous
    vocab_size = 20 * len(_players)


players = [Player(name=name, vocab=pairs) for name in _players]
shuffle(players)

flags = {"japanese": "🇯🇵", "english": "🇬🇧", "aussie": "🇦🇺"}

question = widgets.HTML(
    value="Questions here!",
)

list_players = widgets.Select(
    options=[p.name for p in players],
    value=players[0].name,
    disabled=True
)

progress = widgets.IntProgress(
    value=0,
    min=0,
    max=vocab_size,
    step=1,
    description="[  0 /{:>3}]".format(vocab_size),
    bar_style='',
    orientation='horizontal'
)

status = widgets.ToggleButtons(
    options=['Known ', 'Unkown ', 'Skipped '],
    disabled=False,
    button_style='',
    icons=["check-circle", "question-circle", "forward"],
)
_mapping = {"Known ": "known", "Unkown ": "failed", "Skipped ": "skipped"}

next_b = widgets.Button(
    description='Next!',
    button_style='',
    icon='check'
)

async def f():
    for i in range(vocab_size):
        player_idx = i % len(players)
        curr_player = players[player_idx]
        list_players.value = curr_player.name
        word = curr_player.pick_word()

        q, a = choice([
            ["japanese", "english"],
            ["english", "japanese"],
        ])
        f_q, f_a = q, a
        if curr_player.name == "Dan":
            # Need the right flag
            f_q = "aussie" if q == "english" else f_q
            f_a = "aussie" if a == "english" else f_a

        question.value = 'From <b>{}</b> {} to <b>{}</b> {}, translate:<p><b style="font-size: 24px">{}</b>'.format(
            q, flags[f_q], a, flags[f_a],
            getattr(word, q),
        )
        _ = await wait_for_change(next_b, 'value')
        next_b.disabled = True
        
        curr_player.update(_mapping[status.value])
        
        progress.value += 1
        progress.description = "[{:>3} /{:>3}]".format(progress.value, vocab_size)
        
        # Avoid quick double press
        sleep(0.5)
        status.value = "Known "
        next_b.disabled = False
        
    print("All done")
    
    for p in players:
        print("~"*80)
        print(f"Name: {p.name}")
        p.show_score()

asyncio.ensure_future(f())

display(progress)
display(question)
display(list_players)
display(status)
next_b

## TODO for sentences notebook

* [ ] Better parsing
* [x] Furigana mode?

In [ ]:
# !python3 -m pip install pykakasi
import pykakasi
import re

conv_k = pykakasi.kakasi()
# conv_k = pykakasi.wakati()
# conv_k.setMode('J', 'H')  # J(Kanji) to H(Hiragana)
conv_k.setMode("J","HF") # Japanese to [H(Hiragana)]furigana
conv_k.setMode("s", True)  # add space, default: no separator
# conv_k.setMode("H","aF")
conv = conv_k.getConverter()
# text = "先生だと、きっと年上なんじゃないですか"
text = "空港券は安ければ安いほどいいとは限らない"
converted = conv.do(text)
# converted

def _convert_one_word(word):
    return "<ruby>" + word.replace("[", "<rt>").replace("]", "</rt>") + "</ruby>"

def kanji_to_html_furigana(text, converter):
    text = re.sub('\s+', '', text)
    res = converter.do(text)
    words = re.split('\s+', res)
    return "".join(_convert_one_word(x) for x in words)
    
    

In [ ]:
out = kanji_to_html_furigana(text, conv)
from IPython.core.display import HTML
HTML(f"<b style='font-size: 24px'>{out}</b>")